In [1]:
import pm4py
import sys
sys.path.append('../../TaskExecutionTimeMining/')
from event_log_transformer import *
pandas.set_option('display.max_columns', None)
pandas.set_option('display.max_rows', 200)


In [ ]:
file_path = '../../../data/BPI Challenge 2017.xes'
event_log = pm4py.read_xes(file_path)

parsing log, completed traces ::   0%|          | 0/31509 [00:00<?, ?it/s]

In [ ]:
event_log

In [ ]:
#event_log['concept'] = event_log['concept:name'] + '_' + event_log['lifecycle:transition']
#event_log = event_log[event_log['concept:name'].str.startswith('W_')]

In [ ]:
#event_log[:99999].to_csv('test.csv', index=False, date_format='%Y-%m-%d %H:%M:%S.%f')

In [ ]:
event_log['lifecycle:transition'].unique()

In [ ]:
event_log['concept:name'].unique()

In [ ]:
start_end_event_log = TransformEventLog.start_end_event_log_mult(event_log,
                                                                   start_name_1 = 'start',
                                                                   #start_name_2 = 'resume',
                                                                   #complete_name_3 = 'suspend',
                                                                   complete_name_1 = 'complete', complete_name_2 = 'ate_abort')

In [ ]:
start_end_event_log.groupby(['lifecycle:transition_start', 'lifecycle:transition_complete']).count()

In [ ]:
(start_end_event_log['duration_seconds']/3600).describe()

In [ ]:
(start_end_event_log['duration_seconds']/3600).plot.box()

In [ ]:
start_end_event_log[start_end_event_log['duration_seconds'] > 3000*3600]

In [ ]:
start_end_event_log[start_end_event_log['case:concept:name'] == 'Application_1283264254'].sort_values('time:timestamp_start')

In [ ]:
event_log[event_log['case:concept:name'] == 'Application_1283264254'].sort_values('time:timestamp')

In [ ]:
# Seconds in day
start_end_event_log = TransformEventLog.seconds_in_day(start_end_event_log, 'time:timestamp_start')

In [ ]:
start_end_event_log

In [ ]:
#start_end_event_log[(start_end_event_log['case:concept:name'] == 'Application_949646433')].sort_values('time:timestamp_start')\
#[['case:concept:name', 'concept:name', 'org:resource_start', 'lifecycle:transition_start', 'time:timestamp_start', 'org:resource_complete', 'lifecycle:transition_complete', 'time:timestamp_complete', 'duration_seconds']]

In [ ]:
# Resource counts

resource_count_event_log = TransformEventLog.value_count_per_case(start_end_event_log, 'org:resource_start',
                                                                  timestamp_name = 'time:timestamp_start',
                                                                 lifecycle_col_name = 'lifecycle:transition_start')

In [ ]:
resource_count_event_log

In [ ]:
activity_count_event_log = TransformEventLog.value_count_per_case(resource_count_event_log, 'concept:name',
                                                                  timestamp_name = 'time:timestamp_start',
                                                                 lifecycle_col_name = 'lifecycle:transition_start' )

In [ ]:
activity_count_event_log

In [ ]:
activity_count_event_log.to_csv('../transformed_event_logs/BPIC_2017_start_end.csv')
activity_count_event_log.to_pickle('../transformed_event_logs/BPIC_2017_start_end.pickle')